In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 例外処理用のライブラリをインポート
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
import sys

import pandas as pd
import numpy as np

seleniumマネジャーが利用可能かハージョン確認  
4.6以前であれば更新

In [12]:
print(selenium.__version__)

4.15.2


## 詳細ページに遷移するための関数
* ~~WebdriverExceptionが発生した場合に待機して、リロードを行う~~<br>
* ~~2間隔でページ全体が更新されたか、最大3秒間確認する~~
* 3秒以上の動画広告に対処するため、この関数ではページ遷移と、エラー発生時のリロードのみ行う。<br>
5回リロードしても同じエラーが発生する場合には強制終了を行う

In [13]:
def Safe_navigation(href, i=0):
  try:
    driver.get(href)
    # 例外オブジェクトを e としてキャッチ
  except Exception as e:
    if i >= 5:
      try:
        sys.exit()
      except SystemExit:
        print('ページ遷移不可のため、強制終了')
      raise
    else:
        print(f"Navigation error: {e}")
        i += 1
        Safe_navigation(href, i)

## 都道府県ごとのランキングを取得する関数

In [14]:
# 都道府県ごとのランキングデータフレームを作成する
def Pref_cycle(df_pref):
  for i in range(1, 48):
    # 都道府県ごとのデータ取得
    pref_element = driver.find_element(By.XPATH, '//div[@class="wrap"]/nav/div[@class="left"]/select[@name="state"]')
    # 都道府県のselect要素をselectオブジェクトへ変換
    pref_select = Select(pref_element)
    # オブジェクトからvalue属性のリストを作成する
    prefvalue_list = [pref.get_attribute('value') for pref in pref_select.options]
    # 都道府県の要素をドロップダウンから選択して
    pref_select.select_by_value(prefvalue_list[i])
    # 表示ボタンの要素を取得できるまで最大3秒間待機し、2秒ごとに要素が表示されるか確認
    viewer = WebDriverWait(driver, 3, poll_frequency=2).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
    )
    # 要素（表示ボタン）が表示されるまでスクロールする
    driver.execute_script("arguments[0].scrollIntoView();", viewer)
    viewer.click()

    # 必要な要素(top3の店名とポイント, top4~50の店名とポイント)が表示されるまで最大5秒間待機
    store_ele_top3 = WebDriverWait(driver, 5, poll_frequency=2).until(
      EC.presence_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a'))
    )
    store_top3_point = WebDriverWait(driver, 5).until(
      EC.presence_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="point"]'))
    )
    store_ele = WebDriverWait(driver, 5).until(
      EC.presence_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a'))
    )
    store_point = WebDriverWait(driver, 5).until(
      EC.presence_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td[@class="point"]'))
    )
    # 取得したtop3の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_top3_list = [store_top3.text for store_top3 in store_ele_top3]
    store_point_top3_list = [store_top3.text for store_top3 in store_top3_point]
    href_list_top3 = [ hrefs.get_attribute('href') for hrefs in store_ele_top3]
    
    # 取得したtop4~50の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_list = [store.text for store in store_ele]
    store_point_list = [store.text for store in store_point]
    href_list = [hrefs.get_attribute('href') for hrefs in store_ele]

    # リストをそれぞれ結合する
    store_name_list = store_name_top3_list + store_name_list
    store_point_list = store_point_top3_list + store_point_list
    href_list = href_list_top3 + href_list

    store_detail(store_name_list, store_point_list, href_list, df_pref, prefvalue_list[i])

def Check_namePoint_ele(driver):
  # top3の店のみ店名とポイントを取得
  store_name_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a')
  store_point_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="point"]')
  # top4~50の店名とポイントを取得
  store_name = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a')
  store_point = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td[@class="point"]')
  return store_name_top3, store_point_top3, store_name, store_point
        
#絞込に従う、ランキングtop4~5店の詳細情報を取得する
def store_detail(name_list, points_list, href_list, df_pref, pref_name):
  # top4~50の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    Safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = Get_address()
    # 緯度経度の要素を取得する
    latitude,  longitude = Get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = Get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = Get_holiday()
    Mk_detail_list(name_list[i], points_list[i], address, latitude, longitude, parking, holiday, df_pref, pref_name)
  # 定期的なキャッシュの削除
  driver.get('chrome://settings/clearBrowserData')
  # ランキングのページに戻り、都道府県のプルダウンを取得できるようにする
  driver.get('https://ramendb.supleks.jp/rank')
  # ページ全体がロードされるのを最大3秒間待機する
  WebDriverWait(driver, 3).until(
      lambda d: d.execute_script('return document.readyState') == 'complete'
  )
    
def Get_address():
  try:
    #addressの要素を2秒おきに確認しながら、最大3秒待機する
    address_ele = WebDriverWait(driver, 3, poll_frequency=2).until(
        EC.presence_of_element_located((By.XPATH, '//tbody/tr/td/span[@itemprop="address"]'))
    )
    address = address_ele.text
  # 住所が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    address = None
  return address
        
def Get_map():
  try:
    # mapの要素を確認しながら、最大3秒待機する。addressと同じページ内なので、確認秒数はデフォルトの500ミリ秒とする
    map_data_ele = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.ID, 'gpetitembedmap'))
    )
    map_data = map_data_ele.get_attribute('src')
    # srcの要素値を緯度経度だけ切り離す
    # q= という文字が入るので、スライスで取り除く
    map_data = map_data.split('&')[1][2:]
    # 緯度経度に分割する
    map_data = map_data.split(',')
    latitude = map_data[0]
    longitude = map_data[1]
  # 緯度経度が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    latitude = None
    longitude = None
  return latitude, longitude
        
def Get_parking():
  try:
    th_element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "駐車場")]'))
    )
    parking = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    parking = None
  return parking
        
def Get_holiday():
  try:
    th_element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "定休日")]'))
    )
    holiday = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    holiday = None
  return holiday

# 店舗名とポイント、店舗詳細を店舗ごとにリスト化する
def Mk_detail_list(name, points, address, latitude, longitude, parking, holiday, df_pref, pref_name):
  detail_list = [name]
  detail_list.append(points)
  detail_list.append(pref_name)
  detail_list.append(address)
  detail_list.append(latitude)
  detail_list.append(longitude)
  detail_list.append(parking)
  detail_list.append(holiday)
  print(detail_list)
  df_pref.loc[len(df_pref)] = detail_list

## 分類ごとのランキングを取得する関数

In [15]:
# 旧版：店名とランキングのみ取得

#top3の店名と4〜50の店名の要素をリスト形式で返却する
def Check_name_ele_bk(driver):
    store_name_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a')
    store_name = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a')
    return store_name_top3, store_name

# 分類ごとのランキング表を作成する
def Kind_cycle_bk(kinds_another_list, pager):
    for i in pager:
        # 分類別ごとのデータ取得
        # 分類別、スープ別のselect要素を取得する
        kinds_sel_ele = driver.find_element(By.XPATH, '//select[@name="style"]')
        # 分類別の要素をオブジェクトへ変換する
        kinds_sel_obj = Select(kinds_sel_ele)
        # オブジェクトの、value属性のリストを作成する
        kinds_val_list = [kind.get_attribute('value') for kind in kinds_sel_obj.options]
        kinds_tex_list = [kind.text for kind in kinds_sel_obj.options]
        # 都道府県の要素をドロップダウンから選択して
        kinds_sel_obj.select_by_value(kinds_val_list[i])
        # 表示ボタンの要素を取得できるまで待機（最大十秒）
        viewer = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
        )
        # 要素（表示ボタン）が表示されるまでスクロールする
        driver.execute_script("arguments[0].scrollIntoView();", viewer)
        viewer.click()
        
        # top3の店名と、top4~50の店名を取得できるまで2秒おきに確認しながら、最大3秒待機
        store_name_top3, store_name = WebDriverWait(driver, 3, poll_frequency=2).until(
            Check_name_ele_bk
        )

        # 取得したtop3の店名をリスト形式で保存
        store_name_top3_list = [store_top3.text for store_top3 in store_name_top3]
        # 取得したtop4~50の店名をリスト形式で保存
        store_name_list = [store.text for store in store_name]
        
        kinds_another_list.append(store_name_top3_list + store_name_list)
    
    # 種別ごとのランキングをカラムにするため、numpy形式に変換して転置する
    kinds_another_nplist = np.array(kinds_another_list).T
    return kinds_another_nplist, kinds_tex_list


In [16]:
#新版：都道府県の際と同様のデータを取得。さらにTOP50ではなく、TOP 100まで取得することとする

def Kind_cycle(df_kind, pager):
  for i in pager:
    # 分類別ごとのデータ取得
    # 分類別、スープ別のselect要素を取得する
    kinds_sel_ele = driver.find_element(By.XPATH, '//select[@name="style"]')
    # 分類別の要素をオブジェクトへ変換する
    kinds_sel_obj = Select(kinds_sel_ele)
    # オブジェクトの、value属性のリストを作成する
    kinds_val_list = [kind.get_attribute('value') for kind in kinds_sel_obj.options]
    kinds_text_list = [kind.text for kind in kinds_sel_obj.options]
    # 都道府県の要素をドロップダウンから選択して
    kinds_sel_obj.select_by_value(kinds_val_list[i])
    # 表示ボタンの要素を取得できるまで待機
    viewer = driver.find_element(By.XPATH, '//div[@class="left"]/input[@class="formbtn"]')
    # 要素（表示ボタン）が表示されるまでスクロールする
    driver.execute_script("arguments[0].scrollIntoView();", viewer)
    viewer.click()

    # 要素(top3の店名, top4~50の店名)がページ上に表示されるまで最大10秒待機、その後実行
    store_top3_ele = WebDriverWait(driver, 10, poll_frequency=2).until(
      EC.presence_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a'))
    )
    store_ele = WebDriverWait(driver, 10).until(
      EC.presence_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a'))
    )
    # 取得したtop3の店名、詳細ページのurlをリスト形式で保存
    store_name_top3_list = [store_top3.text for store_top3 in store_top3_ele]
    href_list_top3 = [ hrefs.get_attribute('href') for hrefs in store_top3_ele]
    # 取得したtop4~50の店名、詳細ページのurlをリスト形式で保存
    store_name_list = [store.text for store in store_ele]
    href_list = [hrefs.get_attribute('href') for hrefs in store_ele]

    #top51~100が掲載されたページに遷移する
    driver.execute_script("window.location.href='/rank?page=2&style=1-1';")
    # 要素がページ上に表示されるまで最大5秒待機、その後実行
    store_ele_next_rank = WebDriverWait(driver, 10, poll_frequency=2).until(
      EC.presence_of_all_elements_located((By.XPATH, '//tbody/tr/td/span[@class="name"]/a'))
    )
    store_name_next_rank_list = [store.text for store in store_ele_next_rank]
    href_next_rank_list = [hrefs.get_attribute('href') for hrefs in store_ele_next_rank]
    
    # リストをそれぞれ結合する
    store_names_list = store_name_top3_list + store_name_list + store_name_next_rank_list
    hrefs_list = href_list_top3 + href_list + href_next_rank_list
    Store_detail_of_kind(kinds_text_list[i], store_names_list, hrefs_list, df_kind)

#絞込に従う、ランキングtop4~5店の詳細情報を取得する
def Store_detail_of_kind(kind, name_list, href_list, df_kind):
  # top4~50の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    Safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = Get_address()
    # pointが登録されている要素を取得
    point = Get_point()
    # 緯度経度の要素を取得する
    latitude,  longitude = Get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = Get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = Get_holiday()
    Mk_detail_list_of_kind(kind, name_list[i], point, address, latitude, longitude, parking, holiday, df_kind)
  # 定期的なキャッシュの削除
  driver.get('chrome://settings/clearBrowserData')
  # ランキングのページに戻り、ジャンルのプルダウンを取得できるようにする
  driver.get('https://ramendb.supleks.jp/rank')
  # ページ全体がロードされるのを最大3秒間待機する
  WebDriverWait(driver, 3).until(
      lambda d: d.execute_script('return document.readyState') == 'complete'
  )
    
def Get_point():
  try:
    point_ele_int = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//span[@itemprop="ratingValue"]/span[@class="int"]'))
    )
    point_ele_float = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//span[@itemprop="ratingValue"]/span[@class="float"]'))
    )
    point = point_ele_int.text + point_ele_float.text
  # 住所が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    point = None
  return point

# 店舗名とポイント、店舗詳細を店舗ごとにリスト化する
def Mk_detail_list_of_kind(kind, name, point, address, latitude, longitude, parking, holiday, df_kind):
  detail_list = [kind]
  detail_list.append(name)
  detail_list.append(point)
  detail_list.append(address)
  detail_list.append(latitude)
  detail_list.append(longitude)
  detail_list.append(parking)
  detail_list.append(holiday)
  print(detail_list)
  df_kind.loc[len(df_kind)] = detail_list

## 都道府県ごとのランキングを取得したい場合に実行する

In [43]:
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(3)
driver.get('https://ramendb.supleks.jp/rank')

# # データフレームの枠組みを作成する
columns_list_detail = ['kind', 'name', 'point', 'pref', 'address', 'latitude', 'longitude', 'parking', 'holiday']
df_pref = pd.DataFrame(columns = columns_list_detail)

Pref_cycle(df_pref)
df_pref.to_csv('../data/pref_rank.csv',index=False)

NameError: name 'select_by_value' is not defined

## 種類ごとのランキングが取得したい場合に実行する

In [83]:
# 旧版
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(5)
driver.get('https://ramendb.supleks.jp/rank')

# 種別ごとのTOP50の店名のリストを格納するリストを作成
kinds_another_list = []
# 実行するプルダウンのリスト
pager = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16]

kinds_another_nplist, kinds_tex_list = Kind_cycle_bk(kinds_another_list, pager)

kinds_tex_list.remove('全て')
while "その他" in kinds_tex_list:
    kinds_tex_list.remove("その他")

# 取得した種類別のランキングをカラム別にcsv形式に変換
df_kindsrank = pd.DataFrame(kinds_another_nplist, columns = kinds_tex_list)
df_kindsrank.to_csv("../data/kinds_of_rank_bk.csv",index=False)

KeyboardInterrupt: 

In [18]:
# 新版
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(20)
driver.get('https://ramendb.supleks.jp/rank')

# 実行するプルダウンのリスト
pager = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16]

# # データフレームの枠組みを作成する
columns_list_detail = ['kind', 'name', 'point', 'address', 'latitude', 'longitude', 'parking', 'holiday']
df_kind = pd.DataFrame(columns = columns_list_detail)
Kind_cycle(df_kind, pager)
df_kind.to_csv('../data/kinds_of_rank.csv',index=False)

['ラーメン', '麺屋吉左右', '99.746', '〒135-0016 東京都江東区東陽1-11-3', '35.668467872107', '139.81030784548', '駐車場なし\n周辺にコインパーキング数か所あり', '水曜日・金曜日・日曜日']
['ラーメン', 'らぁ麺 飯田商店 湯河原本店', '99.704', '〒259-0303 神奈川県足柄下郡湯河原町土肥2丁目12-14', '35.144696353473', '139.10973806564', '駐車場あり\n道路向かいの駐車場に5台、斜め向かいに10台\n詳しくはこちら↓\nhttp://photozou.jp/photo/show/1771915/250761120', '火・水曜日\n※不定期月曜臨時休業あり。']
['ラーメン', 'ラーメン二郎 ひばりヶ丘駅前店', '99.403', '〒188-0001 東京都西東京市谷戸町3-27-24 ひばりヶ丘プラザ1F', '35.74995713981', '139.54354908317', '駐車場なし\n近隣にコインパーキング有', '水曜夜の部\n日曜\n※祝日は不定休、営業有無は店内表示やTwitterで告知有り']
['ラーメン', '人類みな麺類', '99.258', '〒532-0011 大阪府大阪市淀川区西中島1-12-15', '34.725506385173', '135.49905161286', '駐車場なし\n近隣にコインパーキングあり', '無休']
['ラーメン', '煮干中華ソバ イチカワ', '99.180', '茨城県つくば市天久保2-9-2 リッチモンド二番街 A101', '36.0982981', '140.1083481', '駐車場あり\nテナントの共同駐車場約8台のみ\n最寄りの有料駐車場は天久保1-10-18付近「ファーストパークつくば天久保第3」…店から南650m、徒歩8分ほど', '基本は、日曜と祝日。\nその他不定休あり（ツイッターにて毎月告知アリ）']
['ラーメン', 'SOBAHOUSE 金色不如帰 新宿御苑本店', '99.013', '〒160-0022 東京都新宿区新宿2-4-1 第22宮庭マンション1F 105\nこのお店は「東京都渋谷区幡ヶ